In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

# If using group_unknown, the output is: X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid
# Otherwise: X_train, Y_train, X_valid, Y_valid

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train

In [8]:
# Helper function
def get_shape_after_skipping(initial_shape, strides_list):
    h, w = initial_shape
    for strides in strides_list:
        h = math.ceil(h / strides[0])
        w = math.ceil(w / strides[1])
    return h, w

In [9]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_concat_clstm import *

In [12]:
graph_name = 'SpectrogramConcatCLSTM'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[-1].shape[1]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

g_cnfg.conv_keep_prob = 0.9
g_cnfg.conv_n_filters = 32
g_cnfg.conv_kernel_size = (10, 10)  # height, width
g_cnfg.conv_mp_size = (2, 1)  # height, width
g_cnfg.conv_mp_strides = (2, 1)  # height, width

g_cnfg.dim_per_time_after_conv = math.ceil(g_cnfg.X_img_h / g_cnfg.conv_mp_strides[0]) * g_cnfg.conv_n_filters

g_cnfg.lstm_state_sizes = [100, 200]
g_cnfg.n_hidden = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 1.0  # 1.0 means no decay

t_cnfg = Config('run_02_group_unknown')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.random_pick_unknown = True
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label(['_background_noise_'])])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0] / t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [ ]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_img_h: 128
X_img_w: 71
Y_vector_len: 12
conv_keep_prob: 0.9
conv_kernel_size: (10, 10)
conv_mp_size: (2, 1)
conv_mp_strides: (2, 1)
conv_n_filters: 32
dim_per_time_after_conv: 2,048
lr_decay_rate: 1.0
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [100, 200]
n_hidden: 50
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8

name: run_02_group_unknown
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 146
random_pick_unknown: 1
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramConcatCLSTM_graph_01_run_02_group_unknown
Training starts @ 01/08/2018 04:00:50
Epoch 1 Step 1 Best model saved @ 01/08/2018 04:00:57 [Ave valid] 2.485
Epoch 1 Step 1 ends @ 01/08/2018 04:00:57 [Train] 2.485, 6.0% [Valid] 10.4% [Ave